## Helper functions to get ground truths and detections

### All the images are stored in 'images' folder. All the .xml files are stored in 'labels' folder 

In [1]:
import os
import tensorflow as tf
import cv2 as cv
from PIL import Image
import seaborn as sns
import numpy as np
import xml.etree.ElementTree as ET
sns.set()

### Load the trained model

The saved_model.pb file contains the weights of the object detection network trained on the given dataset. The weights are stored in a seperate folder 'saved_model'. We have used a pretrained SSD ResNet50 and fine-tuned it on the given dataset

In [3]:
detect_fn = tf.saved_model.load('saved_model')

In [4]:
def boundingbox(im,model,threshold=0.3):
    image_np=np.array(im)
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = model(input_tensor)
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    boxes=list()
    height=image_np.shape[0]
    width=image_np.shape[1]
    for i in range(detections['detection_boxes'].shape[0]):
        box=detections['detection_boxes'][i]
        if detections['detection_scores'][i]>threshold:
            ymin=int(box[0]*height)
            xmin=int(box[1]*width)
            ymax=int(box[2]*height)
            xmax=int(box[3]*width)
            boxes.append([detections['detection_scores'][i],xmin,ymin,xmax,ymax])
    return boxes

In [5]:
def write_to_txt(suffix,boxes,output):
    with open(output+suffix+'.txt', 'w') as f:
        for box in boxes:
            f.write('Table')
            f.write(' ')
            for item in box:
                f.write(str(item))
                f.write(' ')
            f.write('\n')

In [6]:
def xml_writer(tree):
    boxes=list()
    root = tree.getroot()
    for image in root.findall('object'):
        for box in image.findall('bndbox'):
            xmin = int(box.findall('xmin')[0].text)
            ymin = int(box.findall('ymin')[0].text)
            xmax = int(box.findall('xmax')[0].text)
            ymax = int(box.findall('ymax')[0].text)
            boxes.append([xmin,ymin,xmax,ymax])
    return boxes

In [9]:
path = 'images/'
output='detections/'
for f in os.listdir(path):
    suffix = f.split('.')[0]
    im = Image.open(path+f)
    boxes=boundingbox(im,detect_fn)
    write_to_txt(suffix,boxes,output)

In [7]:
path='labels/'
output='groundtruths/'
for f in os.listdir(path):
    suffix = f.split('.')[0]
    tree = ET.parse(path+f)
    boxes=xml_writer(tree)
    write_to_txt(suffix,boxes,output)

Run the following command in the command line terminal to see the mAP@(IOU=0.5) 

#### python pascalvoc.py -gtformat xyrb -detformat xyrb

The 'results' folder contains the results